In [1]:
from argparse import ArgumentParser
import torch
import numpy as np

import pytorch_lightning as pl
import spacetimeformer as stf
import pandas as pd

from pytorch_lightning.loggers import WandbLogger
from data import preprocess
import time
import tqdm
from itertools import chain

from utils import *

/home/vdesai/anaconda3/envs/bats/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class Temp:
    def __init__(self, input_data_path):
        self.input_data_path = input_data_path
        self.pca_components = 0
        
df, max_seq_len = preprocess.preprocess(Temp("/home/vdesai/bats_data/training_files/data.feather"))

In [3]:
g = df.groupby("Filename")

In [3]:
df

,Filename,ParentDir,NextDirUp,TimeInFile,PrecedingIntrvl,CallsPerSec,CallDuration,Fc,HiFreq,LowFreq,...,meanKn-FcCurviness,Kn-FcCurvinessTrndSlp,Path,Version,Filter,Preemphasis,MinAccpQuality,MaxSegLnght,Max#CallsConsidered,TimeIndex
0,barn1_D20220205T192049m784-HiF.wav,0.0,0,0.0,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0,0,0,0,0.0,0,0.0,0
1,barn1_D20220205T192049m784-HiF.wav,0.0,0,0.0,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0,0,0,0,0.0,0,0.0,1
2,barn1_D20220205T192049m784-HiF.wav,0.0,0,0.0,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0,0,0,0,0.0,0,0.0,2
3,barn1_D20220205T192049m784-HiF.wav,0.0,0,0.0,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0,0,0,0,0.0,0,0.0,3
4,barn1_D20220205T192049m784-HiF.wav,0.0,0,0.0,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0,0,0,0,0.0,0,0.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13774067,barn1_D20220723T220206m609.wav,20220723.0,20220720_to_20220723,0.0,206.496,8.283792,6.547682,28.750201,48.257131,27.885463,...,0.000634,-0.000030,Z:\barn1\grouped_audio\20220720_to_20220723\20...,4.4.5,5 kHz,medium,0.8,0.5 sec,32.0,53
13774068,barn1_D20220723T220206m609.wav,20220723.0,20220720_to_20220723,118.0,118.656,8.283792,4.361723,31.419242,47.473494,29.893211,...,0.001330,-0.000938,Z:\barn1\grouped_audio\20220720_to_20220723\20...,4.4.5,5 kHz,medium,0.8,0.5 sec,32.0,54
13774069,barn1_D20220723T220206m609.wav,20220723.0,20220720_to_20220723,281.0,161.712,8.283792,4.414149,29.048323,48.753857,28.884098,...,0.001045,0.004635,Z:\barn1\grouped_audio\20220720_to_20220723\20...,4.4.5,5 kHz,medium,0.8,0.5 sec,32.0,55
13774070,barn1_D20220723T220206m609.wav,20220723.0,20220720_to_20220723,308.0,28.656,8.283792,4.515113,31.079839,48.363437,31.036553,...,0.002251,0.004858,Z:\barn1\grouped_audio\20220720_to_20220723\20...,4.4.5,5 kHz,medium,0.8,0.5 sec,32.0,56


In [ ]:
ignore_cols = ["Filename", "NextDirUp", 'Path', 'Version', 'Filter', 'Preemphasis', 'MaxSegLnght', "ParentDir"]
bats_time_series = stf.data.CSVTimeSeries(
                        raw_df = df,
                        time_col_name = "TimeIndex",
                        time_features = ["hour", "minute", "seconds"],
                        ignore_cols = ignore_cols,
                        val_split = 0.1,
                        test_split = 0.1
                    )

x_dim = bats_time_series.time_cols.size
yc_dim = len(bats_time_series.target_cols)
yt_dim = len(bats_time_series.target_cols)

print(f"{x_dim = }, {yc_dim = }, {yt_dim = }")
